### Script to access and move ICESat-2 data to Pangeo for initial tutorial development

This is a simple test to move one ATL03 dataset to our Pangeo instance for testing and tutorial development

Amy Steiker, NSIDC;
Anthony Arendt, UW

### Import libraries

In [16]:
import getpass
import socket
import requests
import json
import zipfile
import os
import io
import s3fs

### Earthdata login

Store your unique credentials in a separate .cred json file that is in .gitignore and not pushed to GitHub.

In [17]:
with open('../.cred') as json_file:  
    creds = json.load(json_file)

## Request token from Common Metadata Repository using Earthdata credentials

In [18]:
token_api_url = 'https://api.echo.nasa.gov/echo-rest/tokens'
hostname = socket.gethostname()
ip = socket.gethostbyname(hostname)

data = {
    'token': {
        'username': creds['uid'],
        'password': creds['pwd'],
        'client_id': 'NSIDC_client_id',
        'user_ip_address': ip
    }
}
headers={'Accept': 'application/json'}
response = requests.post(token_api_url, json=data, headers=headers)
token = json.loads(response.content)['token']['id']

### A sample API call for a specific region

In [25]:
response = requests.get("https://n5eil02u.ecs.nsidc.org/egi/request? \
                         short_name=ATL06&version=205&temporal=2018-10-17T00:00:00,2019-10-17T23:59:59 \
                         &bounding_box=-121.481,46.7015,-112.036,47.0365&agent=NO&token={}&page_size=10&email={}".format(token,creds['email']))      

### This acquires the data and unzips it

In [10]:
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall(os.getcwd())

### Upload data to S3

After conda installing the awscli, I ran this on the command line to push to the bucket that @Scott created